In [204]:
import pandas as pd
import numpy as np
import string
import re
 
# Load the data
data = pd.read_csv('trial.csv')

In [205]:
data.head(5)

,input,time,date,task_detail
0,Schedule a meeting with John tomorrow at 2 PM.,2 PM,Tomorrow,Meeting with John
1,Remind me to buy groceries on Friday at 6 PM.,6 PM,Friday,Buy groceries
2,Schedule a call with Sarah for next Monday at ...,10 AM,next Monday,Call with Sarah
3,Set a reminder to submit the project report by...,5 PM,tomorrow,Submit the project report
4,Remind me to water the plants on Sunday morning.,morning,Sunday,Water the plants


In [206]:
def remove_punctuation(text):
    # Define a regex pattern to match punctuation
    punctuation_pattern = re.compile('[%s]' % re.escape(string.punctuation))
    # Remove punctuation from the text
    text_no_punctuation = punctuation_pattern.sub('', text)
    return text_no_punctuation

def preprocess_col(col):
    col = col.astype(str).str.lower()
    col = col.apply(remove_punctuation)
    return col


In [207]:
for col in data.columns:
    data[col] = preprocess_col(data[col])
data.head(4)

,input,time,date,task_detail
0,schedule a meeting with john tomorrow at 2 pm,2 pm,tomorrow,meeting with john
1,remind me to buy groceries on friday at 6 pm,6 pm,friday,buy groceries
2,schedule a call with sarah for next monday at ...,10 am,next monday,call with sarah
3,set a reminder to submit the project report by...,5 pm,tomorrow,submit the project report


In [208]:
def find_indices(sentence, day, time, task_detail):
    words = sentence.split()
    indices = [0]*len(words)
    task_detail_words = task_detail.split()
    time_words = time.split()
    date_words = day.split()
    for i, word in enumerate(words):
        if word in date_words:
            indices[i] = 'date'
        elif word in time_words:
            indices[i] = 'time'
        elif word in task_detail_words:
            indices[i] = 'task_detail'
    return indices

find_indices("Schedule a meeting with John tomorrow at 2 PM.", "tomorrow", "2", "meeting with John")

[0, 0, 'task_detail', 'task_detail', 'task_detail', 'date', 0, 'time', 0]

In [209]:
data['new_column'] = data.apply(lambda row: find_indices(row['input'], row['date'], row['time'], row['task_detail']), axis=1)

In [210]:
data.head(2)

,input,time,date,task_detail,new_column
0,schedule a meeting with john tomorrow at 2 pm,2 pm,tomorrow,meeting with john,"[0, 0, task_detail, task_detail, task_detail, ..."
1,remind me to buy groceries on friday at 6 pm,6 pm,friday,buy groceries,"[0, 0, 0, task_detail, task_detail, 0, date, 0..."


In [236]:
data = pd.read_csv('output.csv')

In [239]:
data.head()

,input,time,date,task_detail,new_column
0,schedule a meeting with john tomorrow at 2 pm,2 pm,tomorrow,meeting with john,"[0, 0, 'task_detail', 'task_detail', 'task_det..."
1,remind me to buy groceries on friday at 6 pm,6 pm,friday,buy groceries,"[0, 0, 0, 'task_detail', 'task_detail', 0, 'da..."
2,schedule a call with sarah for next monday at ...,10 am,next monday,call with sarah,"[0, 0, 'task_detail', 'task_detail', 'task_det..."
3,set a reminder to submit the project report by...,5 pm,tomorrow,submit the project report,"[0, 0, 0, 0, 'task_detail', 'task_detail', 'ta..."
4,remind me to water the plants on sunday morning,morning,sunday,water the plants,"[0, 0, 0, 'task_detail', 'task_detail', 'task_..."


In [256]:
indices = data['new_column']
indices = list(map(eval, indices))

In [257]:
def change_indices(indice):
    start_task_flag = True
    start_date_flag = True
    start_time_flag = True
    task_index = []
    date_index = []
    time_index = []
    
    for i in range(len(indice)):
        if indice[i] == 'task_detail':
            if start_task_flag:
                task_index.append('B-' + indice[i])
                date_index.append('O')
                time_index.append('O')
                start_task_flag = False
            else:
                task_index.append('I-' + indice[i])
                date_index.append('O')
                time_index.append('O')
                
        elif indice[i] == 'date':
            if start_date_flag:
                date_index.append('B-' + indice[i])
                time_index.append('O')
                task_index.append('O')
                start_date_flag = False
            else:
                date_index.append('I-' + indice[i])
                time_index.append('O')
                task_index.append('O')

        elif indice[i] == 'time':
            if start_time_flag:
                time_index.append('B-' + indice[i])
                date_index.append('O')
                task_index.append('O')
                start_time_flag = False
            else:
                time_index.append('I-' + indice[i])
                date_index.append('O')
                task_index.append('O')
                
        elif indice[i] == 0:
            task_index.append('O')
            date_index.append('O')
            time_index.append('O')
    
    return task_index, date_index, time_index


In [258]:
combined_labels = [0, 0, 'task_detail', 'task_detail', 'task_detail', 'date', 0, 'time', 'time']

In [259]:
task_index, date_index, time_index = change_indices(combined_labels)

In [272]:
data.iloc[10]['new_column']

"[0, 0, 0, 0, 'task_detail', 0, 'task_detail', 'task_detail', 0, 0, 'time', 'time', 'time', 'time', 'date']"

In [276]:
data.head()

,input,time,date,task_detail,new_column
0,schedule a meeting with john tomorrow at 2 pm,2 pm,tomorrow,meeting with john,"[0, 0, 'task_detail', 'task_detail', 'task_det..."
1,remind me to buy groceries on friday at 6 pm,6 pm,friday,buy groceries,"[0, 0, 0, 'task_detail', 'task_detail', 0, 'da..."
2,schedule a call with sarah for next monday at ...,10 am,next monday,call with sarah,"[0, 0, 'task_detail', 'task_detail', 'task_det..."
3,set a reminder to submit the project report by...,5 pm,tomorrow,submit the project report,"[0, 0, 0, 0, 'task_detail', 'task_detail', 'ta..."
4,remind me to water the plants on sunday morning,morning,sunday,water the plants,"[0, 0, 0, 'task_detail', 'task_detail', 'task_..."


In [278]:
indices[0]

[0, 0, 'task_detail', 'task_detail', 'task_detail', 'date', 0, 'time', 'time']

In [279]:
task_indices = []
date_indices = []
time_indices = []

for i in range(len(indices)):
    task_index, date_index, time_index = change_indices(indices[i])
    task_indices.append(task_index)
    date_indices.append(date_index)
    time_indices.append(time_index)

In [280]:
task_indices[10]

['O',
 'O',
 'O',
 'O',
 'B-task_detail',
 'O',
 'I-task_detail',
 'I-task_detail',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O']

In [281]:
time_indices[10]

['O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-time',
 'I-time',
 'I-time',
 'I-time',
 'O']

In [282]:
len(task_indices)
print(len(task_indices), len(date_indices), len(time_indices))

100 100 100


In [283]:
data['task_indices'] = task_indices
data['date_indices'] = date_indices
data['time_indices'] = time_indices

In [285]:
data.drop(['new_column','date','time','task_detail'], axis=1, inplace=True)

In [286]:
data.head()

,input,task_indices,date_indices,time_indices
0,schedule a meeting with john tomorrow at 2 pm,"[O, O, B-task_detail, I-task_detail, I-task_de...","[O, O, O, O, O, B-date, O, O, O]","[O, O, O, O, O, O, O, B-time, I-time]"
1,remind me to buy groceries on friday at 6 pm,"[O, O, O, B-task_detail, I-task_detail, O, O, ...","[O, O, O, O, O, O, B-date, O, O, O]","[O, O, O, O, O, O, O, O, B-time, I-time]"
2,schedule a call with sarah for next monday at ...,"[O, O, B-task_detail, I-task_detail, I-task_de...","[O, O, O, O, O, O, B-date, I-date, O, O, O]","[O, O, O, O, O, O, O, O, O, B-time, I-time]"
3,set a reminder to submit the project report by...,"[O, O, O, O, B-task_detail, I-task_detail, I-t...","[O, O, O, O, O, O, O, O, O, O, O, B-date]","[O, O, O, O, O, O, O, O, O, B-time, I-time, O]"
4,remind me to water the plants on sunday morning,"[O, O, O, B-task_detail, I-task_detail, I-task...","[O, O, O, O, O, O, O, B-date, O]","[O, O, O, O, O, O, O, O, B-time]"


In [287]:
data.to_csv('set_index.csv', index=False)